In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic1")
sc = SparkContext(conf=conf)

24/12/10 13:29:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
movies_rdd = sc.parallelize([
    (1,("어벤져스", "마블")),
    (2,("슈퍼맨", "DC")),
    (3,("베트맨", "DC")),
    (4,("겨울왕국", "디즈니")),
    (5,("아이언맨", "마블")),
])

attendances_rdd = sc.parallelize([
    (1,("13934592", "KR")),
    (2,("2182227", "KR")),
    (3,("4226242", "KR")),
    (4,("10303058", "KR")),
    (5,("4300365", "KR")),
])

In [5]:
movies_att = movies_rdd.join(attendances_rdd)

movies_att.filter(
     lambda x: int(x[1][1][0]) > 5000000
).collect()


[(4, (('겨울왕국', '디즈니'), ('10303058', 'KR'))),
 (1, (('어벤져스', '마블'), ('13934592', 'KR')))]

In [8]:
filtered_att = attendances_rdd.filter(lambda x: int(x[1][0]) > 5000000)
movies_rdd.join(filtered_att).collect()


[(4, (('겨울왕국', '디즈니'), ('10303058', 'KR'))),
 (1, (('어벤져스', '마블'), ('13934592', 'KR')))]

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_sql_basic").getOrCreate()

In [10]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [11]:
movie_schema = ['id', 'name', 'company', 'year', 'month', 'day']

In [13]:
df =spark.createDataFrame(data=movies, schema= movie_schema)

In [14]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [15]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [16]:
df.filter(df.year>=2000).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [18]:
df.select('name').show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [20]:
df.createOrReplaceTempView('movies')

In [22]:
query = '''
select name from movies
'''

spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [23]:
# 2010년 이후 개봉한 영화 조회

query = '''
select name, company
from movies
where year>=2010'''

In [24]:
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|겨울왕국| 디즈니|
+--------+-------+



In [ ]:
where company = '마블'

In [ ]:
where company = '마블' company = 'DC'


In [ ]:
# 개봉 연도 오름차순으로 확인
from movies
order by year ASC

In [ ]:
# count(*),  count(name)
# mean, sum
select count(*)

In [34]:
attendances = [
    [1, 13934592., "KR"],
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [35]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [36]:
att_df = spark.createDataFrame( data=attendances, schema = att_schema )
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [37]:
att_df.show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [38]:
att_df.createOrReplaceTempView('att')

In [39]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [40]:
query = '''
select *
from movies JOIN att ON  movies.id = att.id
'''

In [41]:
spark.sql(query).show()

+---+--------+-------+----+-----+---+---+-----------+---------------+
| id|    name|company|year|month|day| id|        att|theater_country|
+---+--------+-------+----+-----+---+---+-----------+---------------+
|  5|아이언맨|   마블|2008|    4| 30|  5|  4300365.0|             KR|
|  1|어벤져스|   마블|2012|    4| 26|  1|1.3934592E7|             KR|
|  3|  배트맨|     DC|2008|    8|  6|  3|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|  4|1.0303058E7|             KR|
+---+--------+-------+----+-----+---+---+-----------+---------------+



In [42]:
query = '''
select movies.id, movies.name, movies.company, att.att, att.theater_country
from movies JOIN att ON  movies.id = att.id
'''

In [43]:
spark.sql(query).show()

+---+--------+-------+-----------+---------------+
| id|    name|company|        att|theater_country|
+---+--------+-------+-----------+---------------+
|  5|아이언맨|   마블|  4300365.0|             KR|
|  1|어벤져스|   마블|1.3934592E7|             KR|
|  3|  배트맨|     DC|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|1.0303058E7|             KR|
+---+--------+-------+-----------+---------------+



In [44]:
spark.stop()